In [32]:
import pandas as pd
import numpy as np

In [33]:
# bring in testing data
test_df = pd.read_csv('DB/test_sample.csv', index_col='Unnamed: 0')
test_df = test_df.dropna()

In [34]:
# create base training features from existing testing features
X = test_df.dropna()

In [35]:
X

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance
0,2020-01-01,WN,WN,WN,4541,WN,N406WN,4541,14679,SAN,"San Diego, CA",12889,LAS,"Las Vegas, NV",1625,1735,N,70,1,258
1,2020-01-01,WN,WN,WN,4566,WN,N494WN,4566,14679,SAN,"San Diego, CA",12889,LAS,"Las Vegas, NV",2105,2215,N,70,1,258
2,2020-01-01,WN,WN,WN,4877,WN,N762SW,4877,14679,SAN,"San Diego, CA",12889,LAS,"Las Vegas, NV",2215,2325,N,70,1,258
3,2020-01-01,WN,WN,WN,5106,WN,N7828A,5106,14679,SAN,"San Diego, CA",12889,LAS,"Las Vegas, NV",1015,1135,N,80,1,258
4,2020-01-01,WN,WN,WN,6366,WN,N287WN,6366,14679,SAN,"San Diego, CA",12889,LAS,"Las Vegas, NV",855,1015,N,80,1,258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197555,2020-01-31,DL,DL_CODESHARE,DL,4820,9E,N294PQ,4820,12953,LGA,"New York, NY",14122,PIT,"Pittsburgh, PA",1736,1924,N,108,1,335
197556,2020-01-31,DL,DL_CODESHARE,DL,4820,9E,N294PQ,4820,14122,PIT,"Pittsburgh, PA",12953,LGA,"New York, NY",1959,2136,N,97,1,335
197557,2020-01-31,DL,DL_CODESHARE,DL,4821,9E,N918XJ,4821,14814,SHV,"Shreveport, LA",10397,ATL,"Atlanta, GA",600,852,N,112,1,551
197558,2020-01-31,DL,DL_CODESHARE,DL,4822,9E,N8946A,4822,12953,LGA,"New York, NY",14576,ROC,"Rochester, NY",1710,1839,N,89,1,254


In [36]:
# find numeric and categorical features
cols = X.columns
num_cols = X._get_numeric_data().columns
cat_cols = list(set(cols) - set(num_cols))

# remove redundant numeric columns
final_num_cols = list(num_cols)
final_num_cols.remove('op_carrier_fl_num')
final_num_cols.remove('flights')

# remove redundant categorical columns
final_cat_cols = ['mkt_unique_carrier', 'fl_date', 'tail_num', 'branded_code_share']

# combine final features
final_features = final_num_cols + final_cat_cols

X = X[final_features]

In [38]:
# convert fl_date feature into datetime
X['fl_date'] = pd.to_datetime(X['fl_date'])

# separate datetime into date features
X['year'] = X['fl_date'].dt.year
X['month'] = X['fl_date'].dt.month
X['week'] = X['fl_date'].dt.isocalendar().week
X['day'] = X['fl_date'].dt.day
X['day_of_week'] = X['fl_date'].dt.dayofweek

# drop original fl_date and arr_delay columns
X = X.drop(columns=['fl_date'])

KeyError: 'fl_date'

In [39]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 197074 entries, 0 to 197559
Data columns (total 15 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   mkt_carrier_fl_num  197074 non-null  int64 
 1   origin_airport_id   197074 non-null  int64 
 2   dest_airport_id     197074 non-null  int64 
 3   crs_dep_time        197074 non-null  int64 
 4   crs_arr_time        197074 non-null  int64 
 5   crs_elapsed_time    197074 non-null  int64 
 6   distance            197074 non-null  int64 
 7   mkt_unique_carrier  197074 non-null  object
 8   tail_num            197074 non-null  object
 9   branded_code_share  197074 non-null  object
 10  year                197074 non-null  int64 
 11  month               197074 non-null  int64 
 12  week                197074 non-null  UInt32
 13  day                 197074 non-null  int64 
 14  day_of_week         197074 non-null  int64 
dtypes: UInt32(1), int64(11), object(3)
memory usage: 23

## Join New Features

In [40]:
# bring in feature data
taxi_and_delay = pd.read_csv('James/features_created.csv', index_col='order')
avg_monthly_pas = pd.read_csv('Riley/avg_monthly_pas.csv')

In [41]:
# drop extra column
taxi_and_delay = taxi_and_delay.drop(columns=['Unnamed: 0'])

In [42]:
# merge onto training DataFrame
final = pd.merge(X, avg_monthly_pas, how='left', on=['origin_airport_id','month'])

In [43]:
final

,mkt_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,mkt_unique_carrier,tail_num,branded_code_share,year,month,week,day,day_of_week,avg_monthly_pas
0,4541,14679,12889,1625,1735,70,258,WN,N406WN,WN,2020,1,1,1,2,76419.0
1,4566,14679,12889,2105,2215,70,258,WN,N494WN,WN,2020,1,1,1,2,76419.0
2,4877,14679,12889,2215,2325,70,258,WN,N762SW,WN,2020,1,1,1,2,76419.0
3,5106,14679,12889,1015,1135,80,258,WN,N7828A,WN,2020,1,1,1,2,76419.0
4,6366,14679,12889,855,1015,80,258,WN,N287WN,WN,2020,1,1,1,2,76419.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197069,4820,12953,14122,1736,1924,108,335,DL,N294PQ,DL_CODESHARE,2020,1,5,31,4,66470.5
197070,4820,14122,12953,1959,2136,97,335,DL,N294PQ,DL_CODESHARE,2020,1,5,31,4,37516.0
197071,4821,14814,10397,600,852,112,551,DL,N918XJ,DL_CODESHARE,2020,1,5,31,4,2378.5
197072,4822,12953,14576,1710,1839,89,254,DL,N8946A,DL_CODESHARE,2020,1,5,31,4,66470.5


In [44]:
final = final.dropna()
final

,mkt_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,mkt_unique_carrier,tail_num,branded_code_share,year,month,week,day,day_of_week,avg_monthly_pas
0,4541,14679,12889,1625,1735,70,258,WN,N406WN,WN,2020,1,1,1,2,76419.0
1,4566,14679,12889,2105,2215,70,258,WN,N494WN,WN,2020,1,1,1,2,76419.0
2,4877,14679,12889,2215,2325,70,258,WN,N762SW,WN,2020,1,1,1,2,76419.0
3,5106,14679,12889,1015,1135,80,258,WN,N7828A,WN,2020,1,1,1,2,76419.0
4,6366,14679,12889,855,1015,80,258,WN,N287WN,WN,2020,1,1,1,2,76419.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197069,4820,12953,14122,1736,1924,108,335,DL,N294PQ,DL_CODESHARE,2020,1,5,31,4,66470.5
197070,4820,14122,12953,1959,2136,97,335,DL,N294PQ,DL_CODESHARE,2020,1,5,31,4,37516.0
197071,4821,14814,10397,600,852,112,551,DL,N918XJ,DL_CODESHARE,2020,1,5,31,4,2378.5
197072,4822,12953,14576,1710,1839,89,254,DL,N8946A,DL_CODESHARE,2020,1,5,31,4,66470.5
